As usual, first we import the necessary libraries.  In particular, we use [Seaborn](http://stanford.edu/~mwaskom/software/seaborn/) to give us a nicer default color palette, with our plots being of large (`poster`) size and with a white-grid background. 

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time

In [2]:
zip_data = pd.read_csv("AfterGotZips2016.csv", 
                      encoding = "ISO-8859-1")
zip_data["BEGIN_ZIP"].head()

# # Get rid of ZIP codes just for PO boxes
# zip_data = zip_data.loc[zip_data['Type'] == 'Standard']
# zip_data = zip_data.reset_index(drop=True)

# zips = zip_data["ZIP Code"]

# for i in range(0, len(zips)):
#     zips[i] = str(zips[i])[9:]
# #print(len(zips))

0    29532
1    29565
2    29543
3    29565
4    29592
Name: BEGIN_ZIP, dtype: object

In [3]:
import requests

s = requests.session()
#s.headers['user-agent'] = 'Mozilla/5.0'
print(s) 
r = s.get('https://factfinder.census.gov/rest/communityFactsNav/nav', params={'searchTerm': 99501})
r
type(r.json())

dict

In [4]:
r.json().keys()
r.json()['CFMetaData'].keys()

dict_keys(['isNotValidGeo', 'displayNoDataAvailableMsg', 'eusbreadcrumb', 'currentContext', 'breadcrumbTitle', 'currentContextURI', 'backToBreadcrumbTitle', 'measureAndLinksContent', 'leftNavSelection', 'disambiguationContent'])

In [5]:
r.json()['CFMetaData']['measureAndLinksContent']

'<div>\n<h2>99501</h2>\n<div class=\'datapoint\'>\n<div class="actionbar">\n<div class="buttons">\n<a id="bookmark_btn" class="ab-bookmark" href="javascript:openCFBookmark();" onmouseover="ab_in(this)" onmouseout="ab_out(this)"><span></span> Bookmark/Save</a>\n<img src="/common/img/button_sep.gif" alt="" width="3" height="18" />\n<a id="print_btn" class="ab-print" href="javascript:printCF();" onmouseover="ab_in(this)" onmouseout="ab_out(this)"><span></span> Print</a>\n</div>\n</div>\n<h3>\nPopulation  </h3>\n<div style="margin:0 0 10px 0">\n<select onchange="selectMeasure(\'POPULATION\', this);" style="font-size:93%">\n<option value="DECENNIAL_CNT"  selected="selected">Census 2010 Total Population</option><option value="PEP_EST" >2016 Population Estimate (as of July 1, 2016)</option><option value="ACS_EST" >2015 ACS 5-Year Population Estimate</option></select>\n</div>\n  <div class=\'value\'>17,603 </div>\n<div class=\'source\'>Source: <a href="http://factfinder.census.gov/bkmk/table/1

In [6]:
html = r.json()['CFMetaData']['measureAndLinksContent']

In [7]:
s.get('https://factfinder.census.gov/bkmk/table/1.0/en/DEC/10_DP/DPDP1/8600000US99501')
r = s.get('https://factfinder.census.gov/tablerestful/tableServices/renderProductData')
r.json().keys()
#['ProductData']
r.json()['ProductData'].keys()

dict_keys(['displayID', 'displayLabel', 'productDataset', 'eusbreadcrumb', 'currentContext', 'breadcrumbTitle', 'currentContextURI', 'backToBreadcrumbTitle', 'universe', 'tableToolsAvailable', 'tableToolsEnabled', 'mappable', 'statsigSupported', 'statsigEnabled', 'disableCSVForNP', 'downloadOnly', 'showHideEnabled', 'previousProductId', 'previousProductType', 'currentProductIndex', 'currentProductType', 'nextProductId', 'nextProductType', 'currentProductHasActiveMap', 'mapActiveInSession', 'totalProductsInSelection', 'renderForMap', 'renderForChart', 'isDocument', 'isNP', 'productRecSpec', 'totalColumnCount', 'presentationDownloadColLimit', 'totalRowCount', 'chartRendered', 'presentationDownloadRowLimit', 'productDataTable', 'requestRejected', 'requestRejectionMessage', 'noDataFound', 'viewAllLimit', 'tableModified', 'resetGeoOverlay', 'headerNotes', 'footerNotes', 'layoutHeaderNotes', 'layoutFooterNotes'])

In [8]:
import requests
s = requests.session()
s.headers['user-agent'] = 'Mozilla/5.0'

s.get('https://factfinder.census.gov/bkmk/table/1.0/en/DEC/10_DP/DPDP1/8600000US99501')
#<Response [200]>
r = s.get('https://factfinder.census.gov/tablerestful/tableServices/renderProductData')
r.json().keys()
#['ProductData']
r.json()['ProductData']['productDataTable'][:50]
#"<div class='actionbar'><div id='notes-div1' class="

"<div class='actionbar'><div id='notes-div1' class="

In [9]:
import requests
from   bs4 import BeautifulSoup
from IPython.core.display import display, HTML

# This is just to get the column names

zipcodes = ['32963']

base   = 'https://factfinder.census.gov/'
report = base + 'bkmk/table/1.0/en/DEC/10_DP/DPDP1/8600000US'
render = base + 'tablerestful/tableServices/renderProductData'

with requests.session() as s:

    for zipcode in zipcodes:
        s.get(report + zipcode)
        r = s.get(render)

        html = r.json()['ProductData']['productDataTable']
        soup = BeautifulSoup(html, 'html.parser')
        
        rows = soup.find("table").find_all("tr")[2:217]
        
        my_cols = []
        
        for row in rows:
            headers = row.find_all("th")
            for header in headers:
                actual_header = header.get_text()
                count = actual_header + "count"
                perc = actual_header + "percent"
                my_cols.append(count)
                my_cols.append(perc)

In [10]:
# my_cols_df = pd.DataFrame(my_cols)
# my_cols_df

# my_data_df = pd.DataFrame(my_data)
# #my_data_df

# my_cols_df["32963"] = my_data
# my_cols_df

In [11]:
unknown_counter = 0

In [12]:
print(len(zip_data["BEGIN_ZIP"][24256:]))
print(len(zip_data["BEGIN_ZIP"]))

1009
25265


In [13]:
# This part is just extracting data then


zipcodes = zip_data["BEGIN_ZIP"][24256:]

scraped = pd.DataFrame()

base   = 'https://factfinder.census.gov/'
report = base + 'bkmk/table/1.0/en/DEC/10_DP/DPDP1/8600000US'
render = base + 'tablerestful/tableServices/renderProductData'

with requests.session() as s:
    #s.headers['user-agent'] = 'Mozilla/5.0'

    for zipcode in zipcodes:
        #print(zipcode)
        
        if zipcode == "Unknown":
            unknown_counter = unknown_counter + 1
        
        else:
            
            time.sleep(3)
            s.get(report + zipcode)
            r = s.get(render)

            html = r.json()['ProductData']['productDataTable']
            soup = BeautifulSoup(html, 'html.parser')
            #print(soup)

            rows = soup.find("table").find_all("tr")[2:217]
            #rows = soup.find("table").find_all("tr")[2]
            #print(rows)
            #headings = [tr.get_text() for th in soup.find("table").find_all("tr").find_all("tr")]
            #print(headings)

            my_cols = []
            my_data = []
            data_zipcode = {}

            for row in rows:
                #print("inhere")
                #print(row)
                headers = row.find_all("th")
                for header in headers:
                    actual_header = header.get_text()

                    count = actual_header + "count"
                    perc = actual_header + "percent"

                    my_cols.append(count)
                    my_cols.append(perc)



                    #print(header.get_text())
                #print(header)

                datums = row.find_all("td") # + row.find_all("th")
                #print(datums)

                actual_data = []

                for datum in datums:
                    #print(datum)
                    actual_data.append(datum.get_text())
                    my_data.append(datum.get_text())
                    #print(datum.get_text())
                #print(actual_data)

                data_zipcode[actual_header] = actual_data
            #print(my_data)
            #print(zipcode)
            my_data = [zipcode] + my_data
            #print(my_data)
            scraped[zipcode] = my_data
    #         print(rows)
    #         datums = rows.find_all("td")[0]
    #         print('now datums')
    #         print(datums.get_text())

            #display(HTML(soup.prettify()))
        
#print('the end')

#data_zipcode
#my_cols

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [ ]:
scraped.shape

In [ ]:
if my_cols[0] != "Zipcode":
    my_cols = ["Zipcode"] + my_cols
better_scraped = scraped.transpose()
better_scraped
print(len(my_cols))
print(better_scraped.shape)

In [ ]:
better_scraped.columns = my_cols
better_scraped

In [ ]:
better_scraped.to_csv("32_457_MonPM2010.csv")

In [ ]:
better_scraped_remove_r = pd.read_csv("ScrapedZipCodeInfo.csv", encoding = "ISO-8859-1")
better_scraped_remove_r.head()

In [ ]:
better_scraped_remove_r.to_csv("ScrapedZipCodeInfo_NoR.csv")

In [ ]:
example = "308,745,538(r38234)"
example.split("(")[0]

In [ ]:
better_scraped_remove_r["Total populationcount"][58:68]

In [ ]:
file = open("laborforce_annual_2015.txt", "r") 
file.read() 

.
.

.
.

.
.


.
.

.

.

.




.
.

.
.

.
.

.
.

.


.


.
.

.

.
.
.
.
.
.
..

## Q1. Scraping Wikipedia for Billboard Top 100.

In this question you will scrape Wikipedia for the Billboard's top 100 singles.

### Scraping Wikipedia for Billboard singles

We'll be using  [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/), and suggest that you use Python's built in `requests` library to fetch the web page.

#### 1.1 Parsing the Billboard Wikipedia page for 1970

Obtain the web page at http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1970 using a HTTP GET request. From this web page we'll extract the top 100 singles and their rankings. Create a list of dictionaries, 100 of them to be precise, with entries like 

`{'url': '/wiki/Sugarloaf_(band)', 'ranking': 30, 'band_singer': 'Sugarloaf', 'title': 'Green-Eyed Lady'}`. 

If you look at that web page, you'll see a link for every song, from which you can get the `url` of the singer or band. We will use these links later to scrape information about the singer or band. From the listing we can also get the band or singer name `band_singer`, and `title` of the song.

*HINT: look for a table with class `wikitable`.*

You should get something similar to this (where songs is the aforementioned list):

```
songs[2:4]
```

```
[{'band_singer': 'The Guess Who',
  'ranking': 3,
  'title': '"American Woman"',
  'url': '/wiki/The_Guess_Who'},
 {'band_singer': 'B.J. Thomas',
  'ranking': 4,
  'title': '"Raindrops Keep Fallin\' on My Head"',
  'url': '/wiki/B.J._Thomas'}]
```

In [ ]:
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests

In [ ]:
# here we access the webpage and download the content using requests
t1970=requests.get("http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1970")
t1970

In [ ]:
# We'll just reuse the request object that was previously created to create a BeautifulSoup element.
soup = BeautifulSoup(t1970.text, "html.parser")

# In this line we are looking for a single "table" element with a class of wikitable;
# and then looking for all the "tr" elements on that table (notice the find vs find_all calls).
rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[1:]

# We then define a function whose job it is to act on
# each column's element in each row in the table.
def cleaner(r):
    ranking = int(r[0].get_text())
    title = r[1].get_text()
    band_singer = r[2].get_text()
    url = r[2].find("a").get("href")
    return [ranking, title, band_singer, url]


# Next we'll create a list of names that will be used as dictionary keys.
fields = ["ranking", "title", "band_singer", "url"]

# We now use the cleaner function to process each "td" element on a given row.
# It gives us a bunch of band information
# The zip function creates a list of pairs; which the dict function then uses
# to create a dictionary, using the first element of the pair as the key and the second as
# the value; and finally, the list comprehension iterates over each row element, and puts
# the result of each iteration on a list, which is then bound to the songs variable.
songs = [dict(zip(fields, cleaner(row.find_all("td")))) for row in rows]

In [ ]:
songs[2:4]

#### 1.2 Generalize the previous: scrape Wikipedia from 1992 to 2014

By visiting the urls similar to the ones for 1970, we can obtain the billboard top 100 for the years 1992 to 2014. (We choose these later years rather than 1970 as you might find music from this era more interesting.) Download these using Python's `requests` module and store the text from those requests in a dictionary called `yearstext`. This dictionary ought to have as its keys the years (as integers from 1992 to 2014), and as values corresponding to these keys the text of the page being fetched.

You ought to sleep a second (look up `time.sleep` in Python) at the very least in-between fetching each web page: you do not want Wikipedia to think you are a marauding bot attempting to mount a denial-of-service attack.

*HINT: you might find `range` and string-interpolation useful to construct the URLs *.


In [ ]:
#your code here
years=range(1992, 2015)
print(years)
yearstext={}
for y in years:
    print(y)
    yreq=requests.get("http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_%i" % y)
    yearstext[y]=yreq.text
    time.sleep(1)

#### 1.3 Parse and Clean data 

Remember the code you wrote to get data from 1970 which produces a list of dictionaries, one corresponding to each single.  Now write a function `parse_year(the_year, yeartext_dict)` which takes the year, prints it out, gets the text for the year from the just created `yearstext` dictionary, and return a list of dictionaries for that year, with one dictionary for each single. Store this list in the variable `yearinfo`.

The dictionaries **must** be of this form:

```
{'band_singer': ['Brandy', 'Monica'],
  'ranking': 2,
  'song': ['The Boy Is Mine'],
  'songurl': ['/wiki/The_Boy_Is_Mine_(song)'],
  'titletext': '" The Boy Is Mine "',
  'url': ['/wiki/Brandy_Norwood', '/wiki/Monica_(entertainer)']}
```

The spec of this function is provided below:

In [ ]:
"""
Function
--------
parse_year

Inputs
------
the_year: the year you want the singles for
yeartext_dict: a dictionary with keys as integer years and values the downloaded web pages 
    from wikipedia for that year.
   
Returns
-------

a list of dictionaries, each of which corresponds to a single and has the
following data:

Eg:

{'band_singer': ['Brandy', 'Monica'],
  'ranking': 2,
  'song': ['The Boy Is Mine'],
  'songurl': ['/wiki/The_Boy_Is_Mine_(song)'],
  'titletext': '" The Boy Is Mine "',
  'url': ['/wiki/Brandy_Norwood', '/wiki/Monica_(entertainer)']}
  
A dictionary with the following data:
    band_singer: a list of bands/singers who made this single
    song: a list of the titles of songs on this single
    songurl: a list of the same size as song which has urls for the songs on the single 
        (see point 3 above)
    ranking: ranking of the single
    titletext: the contents of the table cell
    band_singer: a list of bands or singers on this single
    url: a list of wikipedia singer/band urls on this single: only put in the part 
        of the url from /wiki onwards
    

Notes
-----
See description and example above.
"""

#### Helpful notes

Notice that some singles might have multiple songs:

```
{'band_singer': ['Jewel'],
  'ranking': 2,
  'song': ['Foolish Games', 'You Were Meant for Me'],
  'songurl': ['/wiki/Foolish_Games',
   '/wiki/You_Were_Meant_for_Me_(Jewel_song)'],
  'titletext': '" Foolish Games " / " You Were Meant for Me "',
  'url': ['/wiki/Jewel_(singer)']}
```

And some singles don't have a song URL:

```
{'band_singer': [u'Nu Flavor'],
  'ranking': 91,
  'song': [u'Heaven'],
  'songurl': [None],
  'titletext': u'"Heaven"',
  'url': [u'/wiki/Nu_Flavor']}
```

Thus there are some issues this function must handle:

1. There can be more than one  `band_singer` as can be seen above (sometimes with a comma, sometimes with "featuring" in between). The best way to parse these is to look for the urls.
2. There can be two songs in a single, because of the way the industry works: there are two-sided singles. See https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1997 for an example. You can find other examples in 1998 and 1999.
3. The `titletext` is the contents of the table cell, and retains the quotes that Wikipedia puts on the single.
4. If no song anchor is found (see the 24th song in the above url), assume there is one song in the single, set `songurl` to [`None`] and the song name to the contents of the table cell with the quotes stripped (ie `song` is a one-element list with this the `titletext` stripped of its quotes).

As a check, we can do this for 1997. We'll print the first 5 outputs: `parse_year(1997, yearstext)[:5]`

This should give the following. Notice that the year 1997 exercises the edge cases we talked about earlier.

```
[{'band_singer': ['Elton John'],
  'ranking': 1,
  'song': ['Something About the Way You Look Tonight',
   'Candle in the Wind 1997'],
  'songurl': ['/wiki/Something_About_the_Way_You_Look_Tonight',
   '/wiki/Candle_in_the_Wind_1997'],
  'titletext': '" Something About the Way You Look Tonight " / " Candle in the Wind 1997 "',
  'url': ['/wiki/Elton_John']},
 {'band_singer': ['Jewel'],
  'ranking': 2,
  'song': ['Foolish Games', 'You Were Meant for Me'],
  'songurl': ['/wiki/Foolish_Games',
   '/wiki/You_Were_Meant_for_Me_(Jewel_song)'],
  'titletext': '" Foolish Games " / " You Were Meant for Me "',
  'url': ['/wiki/Jewel_(singer)']},
 {'band_singer': ['Puff Daddy', 'Faith Evans', '112'],
  'ranking': 3,
  'song': ["I'll Be Missing You"],
  'songurl': ['/wiki/I%27ll_Be_Missing_You'],
  'titletext': '" I\'ll Be Missing You "',
  'url': ['/wiki/Sean_Combs', '/wiki/Faith_Evans', '/wiki/112_(band)']},
 {'band_singer': ['Toni Braxton'],
  'ranking': 4,
  'song': ['Un-Break My Heart'],
  'songurl': ['/wiki/Un-Break_My_Heart'],
  'titletext': '" Un-Break My Heart "',
  'url': ['/wiki/Toni_Braxton']},
 {'band_singer': ['Puff Daddy', 'Mase'],
  'ranking': 5,
  'song': ["Can't Nobody Hold Me Down"],
  'songurl': ['/wiki/Can%27t_Nobody_Hold_Me_Down'],
  'titletext': '" Can\'t Nobody Hold Me Down "',
  'url': ['/wiki/Sean_Combs', '/wiki/Mase']}]
```

In [ ]:
# your code here

fields = ["ranking", "song", "songurl", "titletext", "band_singer", "url"]

# Helper functions.
def get_cols(row):
    return row.find_all("th") + row.find_all("td")

def break_a(col):
    return list(map(list, zip(*[(a.get("title").strip('"'), a.get("href")) for a in col.find_all("a")]))) \
            or [[col.get_text().strip('"')], [None]]
    
def parse_cols(cols):
    return [int(cols[0].get_text())] + break_a(cols[1]) + [cols[1].get_text()] + break_a(cols[2])

def create_dict(cols):
    return dict(zip(fields, cols))

# Parser function.
def parse_year(year, yearstext):
    soup = BeautifulSoup(yearstext[year], 'html.parser')
    rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[1:]
    return [create_dict(parse_cols(get_cols(row))) for row in rows]

In [ ]:
parse_year(1997, yearstext)[:5]

In [ ]:
yearinfo = {y:parse_year(y, yearstext) for y in years}

#### Save a json file of information from the scraped files

We do not want to lose all this work, so let's save the last data structure we created to disk. That way if you need to re-run from here, you don't need to redo all these requests and parsing. 

DO NOT RERUN THE HTTP REQUESTS TO WIKIPEDIA WHEN SUBMITTING.

*We **DO NOT** need to see these JSON files in your submission!*

In [ ]:
import json

In [ ]:
# DO NOT RERUN THIS CELL WHEN SUBMITTING
fd = open("/tmp/yearinfo.json","w")
json.dump(yearinfo, fd)
fd.close()
del yearinfo

Now let's reload our JSON file into the yearinfo variable, just to be sure everything is working.

In [ ]:
# RERUN WHEN SUBMITTING
# Another way to deal with files. Has the advantage of closing the file for you.
with open("/tmp/yearinfo.json", "r") as fd:
    yearinfo = json.load(fd)

#### 1.4 Construct a year-song-singer dataframe from the yearly information

Let's construct a dataframe `flatframe` from the `yearinfo`. The frame should be similar to the frame below.  Each row of the frame represents a song, and carries with it the chief properties of year, song, singer, and ranking.

![](https://raw.githubusercontent.com/cs109/a-2017/master/hwassets/images/HW1SC1.png)

To construct the dataframe, we'll need to iterate over the years and the singles per year. Notice how, above, the dataframe is ordered by ranking and then year. While the exact order is up to you, note that you will have to come up with a scheme to order the information.

Check that the dataframe has sensible data types. You will also likely find that the year field has become an "object" (Pandas treats strings as generic objects): this is due to the conversion to and back from JSON. Such conversions need special care.

(As an aside, we used the name `flatframe` to indicate that this dataframe is flattened from a hierarchical dictionary structure with the keys being the years.)

Fix any data type issues with `flatframe`. (See Pandas [astype](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html) function.)

We will use this `flatframe` in the next question.

MULTIPLE WAYS TO DO THIS. HERE IS ONE. ANOTHER WAY IS TO PUT YEAR IN EVERY DICTIONARY IN THE LIST, GET RID OF TOP LEVEL DICT, AND THEN EXPAND OUT THE SINGERS AND SONGS (CARTESIAN PRODUCT) AS BELOW.

In [ ]:
yeardict={}
for y in yearinfo.keys():
    yearlist=yearinfo[y] # get list of singles for year
    yearlist2=[]
    for singledict in yearlist:
        singers=singledict['band_singer']
        for i, singer in enumerate(singers):
            songs=singledict['song'] # generally one per single but maybe more
            for j, song in enumerate(songs): # now inside each singer song combination
                nd={}
                nd['band_singer'] = singer
                nd['url'] = singledict['url'][i]
                nd['song'] = song
                nd['songurl'] = singledict['songurl'][j]
                nd['ranking'] = singledict['ranking']
                yearlist2.append(nd)
    yeardict[y]=pd.DataFrame(yearlist2) # one for each year
yearspanel=pd.Panel.from_dict(yeardict, orient="minor") # stack dataframes into a panel
hierframe=yearspanel.to_frame() # flattening leads to a hierarchical index
hierframe.head()

In [ ]:
flatframe = hierframe.reset_index()
flatframe.head()

In [ ]:
flatframe = flatframe.rename(columns={'minor':'year'})
del flatframe['major']
flatframe.head(8)

In [ ]:
flatframe.dtypes

In [ ]:
flatframe.year = flatframe.year.astype(int)
flatframe.dtypes

### Who are the highest quality singers?

Here we show the highest quality singers and plot them on a bar chart.

#### 1.5 Find highest quality singers according to how prolific they are

What do we mean by highest quality? This is of course open to interpretation, but let's define "highest quality" here as the number of times a singer appears in the top 100 over this time period. If a singer appears twice in a year (for different songs), this is counted as two appearances, not one. 

Make a bar-plot of the most prolific singers. Singers on this chart should have appeared at-least more than 15 times. (HINT: look at the docs for the pandas method `value_counts`.)


In [ ]:
prolific=flatframe.url.value_counts()
prolific[prolific > 15].plot(kind="barh");

#### 1.6 What if we used a different metric?

What we would like to capture is this: a singer should to be scored higher if the singer appears higher in the rankings. So we'd say that a singer who appeared once at a higher and once at a lower ranking is a "higher quality" singer than one who appeared twice at a lower ranking. 

To do this, group all of a singers songs together and assign each song a score `101 - ranking`. Order the singers by their total score and make a bar chart for the top 20.


In [ ]:
scores = flatframe.groupby('url').apply(lambda v: np.sum(101 - v['ranking']))
scores= scores.sort_values(ascending=False)
scores[:20].plot(kind="barh");

#### 1.7 Do you notice any major differences when you change the metric?

How have the singers at the top shifted places? Why do you think this happens?

*your answer here*

It seems Lil Wayne appeared in the top 100 many many times (31 times) but got beat out in terms of rankings by both Usher and Mariah Carey. Now we're in a position to answer the question that was posed in the beginning of this assignment -- who is a *better* singer, Mariah Carey or Rihanna? Well, it turns out Rihanna is more *prolific* by our definition, by quite a lot. So based on our analysis we have to give the title to Rihanna. 

## Q2. Scraping and Constructing: Information about Artists, Bands and Genres from Wikipedia

Our next job is to use those band/singer urls we collected under `flatframe.url` and get information about singers and/or bands.

### Scrape information about artists from wikipedia

We wish to fetch information about the singers or groups for all the winning songs in a list of years.

Here we show a function that fetches information about a singer or group from their url on wikipedia. We create a cache object `urlcache` that will avoid redundant HTTP requests (e.g. an artist might have multiple singles on a single year, or be on the list over a span of years). Once we have fetched information about an artist, we don't need to do it again. The caching also helps if the network goes down, or the target website is having some problems. You simply need to run the `get_page` function below again, and the `urlcache` dictionary will continue to be filled.

If the request gets an HTTP return code different from 200, (such as a 404 not found or 500 Internal Server Error) the cells for that URL will have a value of 1; and if the request completely fails (e.g. no network connection) the cell will have a value of 2. This will allow you to analyse the failed requests.

Notice that we have wrapped the call in whats called _an exception block_. We try to make the request. If it fails entirely, or returns a HTTP code thats not 200, we set the status to 2 and 1 respectively.

In [ ]:
urlcache={}


In [ ]:
def get_page(url):
    # Check if URL has already been visited.
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(1)
        # try/except blocks are used whenever the code could generate an exception (e.g. division by zero).
        # In this case we don't know if the page really exists, or even if it does, if we'll be able to reach it.
        try:
            r = requests.get("http://en.wikipedia.org%s" % url)

            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
    return urlcache[url]


We sort the `flatframe` by year, ascending, first. Think why.

In [ ]:
flatframe=flatframe.sort_values('year')
flatframe.head()

#### Pulling and saving the data

In [ ]:
# DO NOT RERUN THIS CELL WHEN SUBMITTING
# Here we are populating the url cache
# subsequent calls to this cell should be very fast, since Python won't
# need to fetch the page from the web server.
# NOTE this function will take quite some time to run (about 30 mins for me), since we sleep 1 second before
# making a request. If you run it again it will be almost instantaneous, save requests that might have failed
# (you will need to run it again if requests fail..see cell below for how to test this)
flatframe["url"].apply(get_page)

You may have to run this function again and again, in case there were network problems. Note that, because there is a "global" cache, it will take less time each time you run it. Also note that this function is designed to be run again and again: it attempts to make sure that there are no unresolved pages remaining. Let us make sure of this: *the sum below should be 0, and the boolean True.*

In [ ]:
# DO NOT RERUN THIS CELL WHEN SUBMITTING
print("Number of bad requests:",np.sum([(urlcache[k]==1) or (urlcache[k]==2) for k in urlcache])) # no one or 0's)
print("Did we get all urls?", len(flatframe.url.unique())==len(urlcache)) # we got all of the urls

Let's save the `urlcache` to disk, just in case we need it again.

In [ ]:
# DO NOT RERUN THIS CELL WHEN SUBMITTING
with open("/tmp/artistinfo.json","w") as fd:
    json.dump(urlcache, fd)
del urlcache

In [ ]:
# RERUN WHEN SUBMITTING
with open("/tmp/artistinfo.json") as json_file:
    urlcache = json.load(json_file)

#### 2.1 Extract information about singers and bands

From each page we collected about a singer or a band, extract the following information:

1. If the page has the text "Born" in the sidebar on the right, extract the element with the class `.bday`. If the page doesn't contain "Born", store `False`.  Store either of these into the variable `born`. We want to analyze the artist's age.

2. If the text "Years active" is found, but no "born", assume a band. Store into the variable `ya` the value of the next table cell corresponding to this, or `False` if the text is not found.

Put this all into a function `singer_band_info` which takes the singer/band url as argument and returns a dictionary `dict(url=url, born=born, ya=ya)`.

The information can be found on the sidebar on each such wikipedia page, as the example here shows:

![sandg](https://raw.githubusercontent.com/cs109/a-2017/master/hwassets/images/sandg.png).



Write the function `singer_band_info` according to the following specification:

Plot a histogram of the age at which singers achieve their top ranking. What conclusions can you draw from this distribution of ages?

*HINT: You will need to do some manipulation of the `born` column, and find the song for which a band or an artist achieves their top ranking. You will then need to put these rows together into another dataframe or array to make the plot.*

In [ ]:
dfb=largedf[largedf.born!=False][['year','born','ranking','url']]
dfb['byear']=dfb.born.apply(lambda x: int(x.split('-')[0]))
byurl=dfb.groupby('url')
frames=[]
for k, v in byurl:
    minr=v.ranking.min()
    frames.append(v[v.ranking==minr])
topscoresdf=pd.concat(frames)
(topscoresdf.year-topscoresdf.byear).hist(bins=np.arange(10,70,2), normed=True);

*your answer here*

The *age of first fame* has an interesting shape, unimodal with a mode in the late 20s with a long tail to the right. This makes sense because there really is a lower age limit to artists but not so much an upper one. So you expect this type of behavior.

#### 2.4 At what year since inception do bands reach their top rankings?

Make a similar calculation to plot a histogram of the years since inception at which bands reach their top ranking. What conclusions can you draw?

In [ ]:
#your code here
dfg=largedf[largedf.born==False]
dfg=dfg[dfg.ya!=False]#drop em!
dfg['start']=dfg.ya.apply(lambda x: int(x[:4]))
frames2=[]
for k, v in dfg.groupby('url'):
    minr=v.ranking.min()
    frames2.append(v[v.ranking==minr])
tsdfg=pd.concat(frames2)
(tsdfg.year-tsdfg.start).hist(bins=20)

*your answer here*

There are many bands (a majority) that hit the top 100 within 8 years of inception. But there are still a large fraction of bands that take over 10 years to hit the top 100. I'm not saying that hitting this is the be-all and end-all, but it's at least an indication. But it's also important to keep the context in mind, many bands aren't together very long so if they were to hit the top-100 they would have to do it within the time that they are together. But all of this information is available on the Wikipedia pages, so we encourage further exploration.